In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import os


driver = webdriver.Chrome()

genres = ["animation", "biography", "history", "musical", "western"]

final_df = pd.DataFrame()

base_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres={}"

save_path = "F:/vscode/GUVI_IMDB_2024/env/Scripts"

for genre in genres:
    print(f"Scraping Genre: {genre}")
    
    url = base_url.format(genre)
    driver.get(url)
    time.sleep(3)

    def click_button():
        try:
            click_more = driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')
            ActionChains(driver).move_to_element(click_more).perform()
            click_more.click()
            time.sleep(3)
            return True
        except:
            return False

    while click_button():
        print("Clicked 'Load More'")

    name, rating, voting, duration = [], [], [], []

    movie_items = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

    for data in movie_items:
        try:
            movie_name = data.find_element(By.XPATH, './div/div/div/div[1]/div[2]/div[1]/a/h3').text.strip()
            movie_rating = data.find_element(By.XPATH, './div/div/div/div[1]/div[2]/span/div/span/span[1]').text.strip()
            movie_voting = data.find_element(By.XPATH, './div/div/div/div[1]/div[2]/span/div/span/span[2]').text.strip()
            movie_duration = data.find_element(By.XPATH, './div/div/div/div[1]/div[2]/div[2]/span[2]').text.strip()
            
            name.append(movie_name)
            rating.append(movie_rating)
            voting.append(movie_voting)
            duration.append(movie_duration)

        except Exception as e:
            print("Error:", e)
            continue

    df = pd.DataFrame({
        'Title': name,
        'Rating': rating,
        'Voting': voting,
        'Duration': duration,
        'Genre': genre
    })

    df['Title'] = df['Title'].str.replace(r'^\d+\.\s*','',regex=True)
    df['Voting'] = df['Voting'].str.replace(r'[\(\)]',"",regex=True)

    df.to_csv(f"{genre}_2024_movies.csv", index=False)
    final_df=pd.concat([final_df,df], ignore_index=True)
    print(f" Saved {genre} movies to CSV.")

final_df.to_csv("All_Genres_2024_movies.csv", index=False)
print("\n All Geners saved")

driver.quit()


Scraping Genre: animation
Clicked 'Load More'
Clicked 'Load More'
Clicked 'Load More'
Clicked 'Load More'
Clicked 'Load More'
Clicked 'Load More'
Clicked 'Load More'
Clicked 'Load More'
Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div/div/div/div[1]/div[2]/div[2]/span[2]"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF764ADCF45+75717]
	GetHandleVerifier [0x00007FF764ADCFA0+75808]
	(No symbol) [0x00007FF7648A8F9A]
	(No symbol) [0x00007FF7648FF4C6]
	(No symbol) [0x00007FF7648FF77C]
	(No symbol) [0x00007FF7648F1D7C]
	(No symbol) [0x00007FF7649273BF]
	(No symbol) [0x00007FF7648F1C46]
	(No symbol) [0x00007FF764927590]
	(No symbol) [0x00007FF76494F39C]
	(No symbol) [0x00007FF764927153]
	(No symbol) [0x00007FF7648F0421]
	(No symbol) [0x00007FF7648F11B3]
	GetHandleVerifi

In [6]:
final_df

,Title,Rating,Voting,Duration,Genre
0,The Wild Robot,8.2,174K,1h 42m,animation
1,Mufasa: The Lion King,6.6,69K,1h 58m,animation
2,Paddington in Peru,6.7,24K,1h 46m,animation
3,Moana 2,6.6,107K,1h 40m,animation
4,Flow,7.9,81K,1h 25m,animation
...,...,...,...,...,...
824,Adiós al amigo,7.4,8,1h 58m,western
825,Hinter dem Eisernen Horizont,7.3,15,1h 28m,western
826,Myall Creek Day of Justice,6.4,17,1h 18m,western
827,Travesty,6.4,7,1h 18m,western


In [26]:
import re

def convert_duration(dur_min):
    dur_min = dur_min.lower().strip()
    hrs = 0
    mins = 0
    hr_match = re.search(r'(\d+)\s*h', dur_min)
    min_match = re.search(r'(\d+)\s*m',dur_min)
    if hr_match:
        hrs = int(hr_match.group(1))
    if min_match:
        mins = int(min_match.group(1))
    return hrs* 60 + mins
    
def convert_votes(con_votes):
    con_votes = con_votes.strip().upper()
    if 'K' in con_votes:
        return int(float(con_votes.replace('K','')) * 1000)
    elif 'M' in con_votes:
        return int(float(con_votes.replace('M','')) * 1000000)
    return int(con_votes)

final_df['Duration'] = final_df['Duration'].apply(convert_duration)
final_df['Voting'] = final_df['Voting'].apply(convert_votes)
final_df.head()

,Title,Rating,Voting,Duration,Genre
0,The Wild Robot,8.2,174000,102,animation
1,Mufasa: The Lion King,6.6,69000,118,animation
2,Paddington in Peru,6.7,24000,106,animation
3,Moana 2,6.6,107000,100,animation
4,Flow,7.9,81000,85,animation


In [27]:
final_df.to_csv("all_geners_cleaned.csv", index=False)

In [28]:
final_df

,Title,Rating,Voting,Duration,Genre
0,The Wild Robot,8.2,174000,102,animation
1,Mufasa: The Lion King,6.6,69000,118,animation
2,Paddington in Peru,6.7,24000,106,animation
3,Moana 2,6.6,107000,100,animation
4,Flow,7.9,81000,85,animation
...,...,...,...,...,...
824,Adiós al amigo,7.4,8,118,western
825,Hinter dem Eisernen Horizont,7.3,15,88,western
826,Myall Creek Day of Justice,6.4,17,78,western
827,Travesty,6.4,7,78,western


In [ ]:

import pandas as pd
from sqlalchemy import create_engine

host = "localhost"
user = "Revathi"
password = "Nithisha_6"
port = 3306
database= "MAT4"
engine=create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

df=pd.read_csv("all_geners_cleaned.csv")

df.to_sql(name='cleaned_movies', con=engine, if_exists='replace', index=False)
print("Data successfully stored in MySql under 'cleaned_movies' table")

query_df = pd.read_sql("SELECT * FROM cleaned_movies", con=engine)
print(query_df)

Data successfully stored in MySql under 'cleaned_movies' table
                            Title  Rating  Voting  Duration      Genre
0                  The Wild Robot     8.2  174000       102  animation
1           Mufasa: The Lion King     6.6   69000       118  animation
2              Paddington in Peru     6.7   24000       106  animation
3                         Moana 2     6.6  107000       100  animation
4                            Flow     7.9   81000        85  animation
..                            ...     ...     ...       ...        ...
824                Adiós al amigo     7.4       8       118    western
825  Hinter dem Eisernen Horizont     7.3      15        88    western
826    Myall Creek Day of Justice     6.4      17        78    western
827                      Travesty     6.4       7        78    western
828                    El Vaquero     8.4      13        93    western

[829 rows x 5 columns]
